In [1]:
import os

cwd = os.getcwd()

print("Current working directory: {0}".format(cwd))

Current working directory: C:\Users\imdpk\Downloads\jp morgan


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import numpy as np

In [3]:
df = pd.read_csv("Task 3 and 4_Loan_Data.csv")
df.head()

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


In [4]:
X = df[['credit_lines_outstanding', 'loan_amt_outstanding', 'total_debt_outstanding',
        'income', 'years_employed', 'fico_score']]
y = df['default']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Logistic Regression
lr_model = LogisticRegression(class_weight='balanced', max_iter=1000)
lr_model.fit(X_train, y_train)
lr_preds = lr_model.predict_proba(X_test)[:,1]

#Random Forest Classifier

rf_model = RandomForestClassifier(random_state = 42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict_proba(X_test)[:,1]

# Evaluate
print("Logistic Regression AUC:", roc_auc_score(y_test, lr_preds))
print("Random Forest AUC:", roc_auc_score(y_test, rf_preds))

#model = rf_model
model = lr_model

Logistic Regression AUC: 0.9999704294341933
Random Forest AUC: 0.9996564596031282


In [9]:
def calculate_expected_loss(borrower_data, model, recovery_rate =0.1):
    
    df=pd.DataFrame([borrower_data])
    pd_value = model.predict_proba(df)[0][1] #probability of default
    ead = borrower_data['loan_amt_outstanding']
    el = ead * pd_value * (1 - recovery_rate)
    return {
        "PD": pd_value,
        "Expected Loss": el
    }

In [10]:
# Example
borrower = {
    'credit_lines_outstanding': 2,
    'loan_amt_outstanding': 3477.43,
    'total_debt_outstanding': 8389.08,
    'income': 70760.64,
    'years_employed': 8,
    'fico_score': 667
}

result = calculate_expected_loss(borrower, model)
print(result)

{'PD': 4.4495927624452836e-13, 'Expected Loss': 1.3925832623919092e-09}
